In [1]:
import os
import json

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('SPOTIPY_CLIENT_ID_CONTAINED_PLAYLISTS_FETCHER'), client_secret=os.getenv('SPOTIPY_CLIENT_SECRET_CONTAINED_PLAYLISTS_FETCHER'))
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

library_account = os.getenv('SPOTIFY_PLUS_SECONDARY_ACCOUNT_USER_ID')

In [ ]:
playlist_ids = []

immediate_to_sort_id = ''
library_to_sort_id = ''

genre_ids = []
archived_mixtape_ids = []
archived_record_ids = []

current_page = sp.user_playlists(library_account)
while current_page:
    for playlist in current_page['items']:
        id = playlist['id']
        playlist_ids.append(id)

        if playlist['name'].startswith('[G]'):
            genre_ids.append(id)
        elif playlist['name'].startswith('[AM]'):
            archived_mixtape_ids.append(id)
        elif playlist['name'].startswith('[AR]'):
            archived_record_ids.append(id)
        elif playlist['name'].startswith('[1]'):
            immediate_to_sort_id = id
        elif playlist['name'].startswith('[2]'):
            library_to_sort_id = id

    if current_page['next']:
        current_page = sp.next(current_page)
    else:
        current_page = None

In [ ]:
def get_simplified_tracks(playlist_id):
    simplified_tracks = []

    current_page = sp.playlist_items(f'spotify:playlist:{playlist_id}', fields='items.track.id,items.track.is_local,items.track.name,next')
    while current_page:
        for item in current_page['items']:
            track = item['track']
            new_track = {}

            new_track['is_local'] = track['is_local']

            if not track['is_local']:
                new_track['id'] = track['id']
            else:
                new_track['id'] = None
            
            new_track['name'] = track['name']
            
            simplified_tracks.append(new_track)

        if current_page['next']:
            current_page = sp.next(current_page)
        else:
            current_page = None
    
    return simplified_tracks

In [2]:
immediate_to_sort_tracks = []
library_to_sort_tracks = []

genre_id_to_tracks = {}
archived_mixtape_id_to_tracks = {}
archived_record_id_to_tracks = {}

In [9]:
class NoneDict(dict):
    def __missing__(self, key):
        return None

def track_json_decoder(track_dict):
    return NoneDict(track_dict)

def playist_json_decoder(playlist_to_track_dict):
    # Why does this work???
    return NoneDict(playlist_to_track_dict)
    
    # none_dict_filtered = {}
    # for genre_id in playlist_to_track_dict:
    #     none_dict_filtered[genre_id] = NoneDict(playlist_to_track_dict[genre_id])
    # return none_dict_filtered

In [10]:
with open("saved_libraries/immediate_to_sort_tracks.json", "r", encoding="utf-8") as file:
    immediate_to_sort_tracks = json.load(file, object_hook=track_json_decoder)
with open("saved_libraries/library_to_sort_tracks.json", "r", encoding="utf-8") as file:
    library_to_sort_tracks = json.load(file, object_hook=track_json_decoder)

with open("saved_libraries/genre_id_to_tracks.json", "r", encoding="utf-8") as file:
    genre_id_to_tracks = json.load(file, object_hook=playist_json_decoder)